## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

ROOT_PATH = os.path.abspath(os.path.join("..", ".."))
SRC_PATH = os.path.join(ROOT_PATH, "src")
TEST_PATH = os.path.join(ROOT_PATH, "test")
sys.path.insert(0, TEST_PATH)
sys.path.insert(0, SRC_PATH)
print(sys.path)

['/Users/bensmidt/Documents/cai/backend/src', '/Users/bensmidt/Documents/cai/backend/test', '/Users/bensmidt/Documents/cai/backend/sample_usage/gcloud', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/bensmidt/Documents/cai/backend/env/lib/python3.11/site-packages', '/Users/bensmidt/Documents/cai/backend/src']


In [3]:
import logging, socket

logger = logging.getLogger()
fhandler = logging.FileHandler(filename="transcoder.log", mode="w")
format_str = f"[%(asctime)s {socket.gethostname()}] %(filename)s:%(funcName)s:%(lineno)s - %(levelname)s: %(message)s"
formatter = logging.Formatter(format_str)
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

In [5]:
from gcloud.transcoder import GCloudTranscoder
from storage.gcloud_blob import GCloudBlob

In [7]:
transcoder = GCloudTranscoder()

## create_mp4_video_trimming_job()

In [11]:
input_blob = GCloudBlob(
    "cai-dev.appspot.com",
    [
        "B9wClqo0ZYYk7P4kpRmgCJAxiCE3",
        "260ca21f-67ce-4884-be34-21ddaf70038f",
        "260ca21f-67ce-4884-be34-21ddaf70038f",
    ],
)
output_blob = GCloudBlob(
    "cai-dev.appspot.com",
    ["B9wClqo0ZYYk7P4kpRmgCJAxiCE3", "testTranscode", "transcode.mp4"],
)
start_second = 12
end_second = 25
audio_stream_settings = {
    "bitrate_bps": 1411000,
    "codec": "aac",
}
video_stream_settings = {
    "height_pixels": 2160,
    "width_pixels": 3840,
    "bitrate_bps": 8899047,
    "frame_rate": 30000 / 1001,
}
container = "mp4"
location = "us-central1"

job_status = transcoder.create_video_trim_job(
    input_blob,
    output_blob,
    start_second,
    end_second,
    video_stream_settings,
    audio_stream_settings,
    container,
    location,
)
print(job_status.name)

projects/461983484560/locations/us-central1/jobs/8856438d-25ba-4db2-882e-05723d2bc621


## create_mp4_audio_trimming_job

In [18]:
input_blob = GCloudBlob(
    "cai-dev.appspot.com",
    [
        "B9wClqo0ZYYk7P4kpRmgCJAxiCE3",
        "260ca21f-67ce-4884-be34-21ddaf70038f",
        "260ca21f-67ce-4884-be34-21ddaf70038f",
    ],
)
output_blob = GCloudBlob(
    "cai-dev.appspot.com",
    ["testTranscode", "transcode"],
)
start_second = 12
end_second = 25
audio_stream_settings = {
    "bitrate_bps": 1411000,
    "codec": "aac",
}
video_stream_settings = {
    "height_pixels": 2160,
    "width_pixels": 3840,
    "bitrate_bps": 8899047,
    "frame_rate": 30000 / 1001,
}
container = "mp4"
location = "us-central1"

job_status = transcoder.create_video_trim_job(
    input_blob,
    output_blob,
    start_second,
    end_second,
    video_stream_settings,
    audio_stream_settings,
    container,
    location,
)
print(job_status.name)

projects/461983484560/locations/us-central1/jobs/913c3118-0a9b-49c9-acad-a5e45b1662af


## job_state()

In [32]:
job_id = os.path.split(job_status.name)[-1]
transcoder.job_state(job_id, location)

'RUNNING'

## Info

In [13]:
from google.cloud.video import transcoder_v1
from time import sleep

job_id = os.path.basename(job_status.name)

while job_status.state != transcoder_v1.types.Job.ProcessingState.SUCCEEDED:
    print(str(job_status.state))
    job_status = GoogleCloudTranscoderManager.get_job(project_id, location, job_id)
    error_message = job_status.error.message
    error_details = job_status.error.details
    print(f"Error message: {error_message}")
    print(f"Error details: {error_details}")
    sleep(3)

1
Error message: No such object: cai-dev.appspot.com/B9wClqo0ZYYk7P4kpRmgCJAxiCE3/260ca21f-67ce-4884-be34-21ddaf70038f
Error details: [type_url: "type.googleapis.com/google.rpc.ErrorInfo"
value: "\n\rInputNotFound\022\031transcoder.googleapis.com\032p\n\007details\022eNo such object: cai-dev.appspot.com/B9wClqo0ZYYk7P4kpRmgCJAxiCE3/260ca21f-67ce-4884-be34-21ddaf70038f"
]


KeyboardInterrupt: 